In [124]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/training_file.json") as f:
    data = pd.DataFrame(js.loads(line) for line in f)
f.close()

with open("/u/pmpande/author_profiling/testing_file.json") as ft:
    data_test = pd.DataFrame(js.loads(line) for line in ft)
ft.close()

In [177]:
import random
rand_train = random.sample(range(0,len(data)),50000)
print(str(len(set(rand_train))))
train = data.iloc[rand_train].copy()

rand_test = random.sample(range(0,len(data_test)),5000)
print(str(len(set(rand_test))))
test = data_test.iloc[rand_test].copy()

50000
5000


In [178]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import re

count = 0
corpus = []
truth = []
for row in train.iterrows():
    corpus.append(row[1]['text'])
    truth.append(row[1]['stars'])
print(count)       
count = 0
corpus_test = []
truth_test = []
for row in test.iterrows():
    corpus_test.append(row[1]['text'])
    truth_test.append(row[1]['stars'])
print(count)

0
0


In [179]:
from nltk.corpus import sentiwordnet as swn
import nltk
from nltk.stem import *
from nltk.wsd import lesk

new = []
count = 0
for row in corpus:
    lst = []
    tokens = nltk.word_tokenize(row)
    tags = nltk.pos_tag(tokens)
    for tag in tags:
        if tag[1] == 'JJ':
            temp = (tag[0],'a')
            lst.append(temp)
        elif tag[1] == 'RB': 
            temp = (tag[0],'r')
            lst.append(temp)

        
    pos = 0
    neg = 0
    score = []
    for word in lst:
        if len(list(swn.senti_synsets(word[0],word[1]))) != 0:
            val = lesk(tokens, word[0])
            word_sent = swn.senti_synset(val.name())
            pos += word_sent.pos_score()
            neg += word_sent.neg_score()
    total = pos+neg
    if total == 0:
        score.append(0)
        score.append(0)
        score.append(1)
    else:
        score.append(pos/total)
        score.append(neg/total)
        obj_val = 1-((pos/total)+(neg/total))
        if obj_val < 0:
            obj_val *= -1
        score.append(obj_val)
    new.append(score)

In [180]:
new_test = []
for row in corpus_test:
    lst = []
    tokens = nltk.word_tokenize(row)
    tags = nltk.pos_tag(tokens)
    for tag in tags:
        if tag[1] == 'JJ':
            temp = (tag[0],'a')
            lst.append(temp)
        elif tag[1] == 'RB': 
            temp = (tag[0],'r')
            lst.append(temp)
        
    pos = 0
    neg = 0
    
    score = []
    for word in lst:
        if len(list(swn.senti_synsets(word[0],word[1]))) != 0:
            val = lesk(tokens, word[0])
            word_sent = swn.senti_synset(val.name())
            pos += word_sent.pos_score()
            neg += word_sent.neg_score()
            
    total = pos+neg
    if total == 0:
        score.append(0)
        score.append(0)
        score.append(1)
    else:
        score.append(pos/total)
        score.append(neg/total)
        score.append(1-((pos/total)+(neg/total)))
    new_test.append(score)

In [181]:
import numpy as np
train_x = np.array(new)
test_x = np.array(new_test)

In [182]:
from sklearn.linear_model import SGDClassifier
from sklearn import tree
clf = MultinomialNB().fit(train_x,truth)
clf1 = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42).fit(train_x,truth)
clf2 = tree.DecisionTreeClassifier().fit(train_x,truth)

In [183]:
predicted = clf.predict(test_x)
print(np.mean(predicted == truth_test))

predicted1 = clf1.predict(test_x)
print(np.mean(predicted1 == truth_test))

predicted2 = clf2.predict(test_x)
print(np.mean(predicted2 == truth_test))

0.421
0.4306
0.421


In [184]:
from sklearn import metrics
targetNames = ['1','2','3','4','5']
print(metrics.classification_report(truth_test, predicted,target_names=targetNames))
print(metrics.classification_report(truth_test, predicted1,target_names=targetNames))
print(metrics.classification_report(truth_test, predicted2,target_names=targetNames))

             precision    recall  f1-score   support

          1       0.00      0.00      0.00       602
          2       0.00      0.00      0.00       424
          3       0.00      0.00      0.00       620
          4       0.00      0.00      0.00      1249
          5       0.42      1.00      0.59      2105

avg / total       0.18      0.42      0.25      5000

             precision    recall  f1-score   support

          1       0.32      0.52      0.40       602
          2       0.00      0.00      0.00       424
          3       0.17      0.06      0.09       620
          4       0.00      0.00      0.00      1249
          5       0.47      0.86      0.61      2105

avg / total       0.26      0.43      0.32      5000

             precision    recall  f1-score   support

          1       0.34      0.38      0.36       602
          2       0.18      0.05      0.08       424
          3       0.23      0.04      0.07       620
          4       0.24      0.09      0

/u/pmpande/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [185]:
print(metrics.confusion_matrix(truth_test, predicted))
print("---------------------------------")
print(metrics.confusion_matrix(truth_test, predicted1))
print("---------------------------------")
print(metrics.confusion_matrix(truth_test, predicted2))

[[   0    0    0    0  602]
 [   0    0    0    0  424]
 [   0    0    0    0  620]
 [   0    0    0    0 1249]
 [   0    0    0    0 2105]]
---------------------------------
[[ 312    0   34    0  256]
 [ 136    0   39    0  249]
 [ 144    0   37    0  439]
 [ 144    0   42    0 1063]
 [ 234    0   67    0 1804]]
---------------------------------
[[ 229   23   15   43  292]
 [  90   21   17   59  237]
 [  97   22   25   68  408]
 [  97   28   24  114  986]
 [ 160   20   26  183 1716]]


In [186]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print(sqrt(mean_squared_error(truth_test, predicted)))
print(sqrt(mean_squared_error(truth_test, predicted1)))
print(sqrt(mean_squared_error(truth_test, predicted2)))

1.8534832073693033
1.7545939701252822
1.7175564037317668
